# Attention

In [ ]:
#| default_exp models.attention

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn as nn
import torch
from torch_lr_finder import LRFinder
from torchinfo import summary
from lightning import LightningModule, Trainer
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from lightning.pytorch.tuner.tuning import Tuner
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from einops import rearrange


from hydra.utils import instantiate
from omegaconf import OmegaConf

from matplotlib import pyplot as plt
import pandas as pd
import math

from nimrod.utils import get_device, set_seed
from nimrod.models.core import Classifier

from pprint import pprint
import logging
from typing import List, Optional, Type


In [ ]:
#| export
logger = logging.getLogger(__name__)
set_seed()

Seed set to 42


## Self-attention

In [ ]:
B, C, H, W = 64, 3, 16, 16
x = torch.randn(B, C, H, W)

In [ ]:
t = x.view(B, C, -1).transpose(1, 2) # B, T, C
t.shape

torch.Size([64, 256, 3])

In [ ]:
k_projection = nn.Linear(C, C)
q_projection = nn.Linear(C, C)
v_projection = nn.Linear(C, C)

In [ ]:
k = k_projection(t)
q = q_projection(t)
v = v_projection(t)
print(k.shape)

torch.Size([64, 256, 3])


In [ ]:
(q@k.transpose(1,2)).shape

torch.Size([64, 256, 256])

In [ ]:
print(k.shape)
k.transpose(1,0).shape

torch.Size([64, 256, 3])


torch.Size([256, 64, 3])

In [ ]:
#| export

class SelfAttention(nn.Module):
    """
    Self-Attention layer for image data.

    Parameters
    ----------
    n_features : int
        Number of input features (channels).
    Methods
    -------
    forward(x: torch.Tensor) -> torch.Tensor
        Forward pass through the self-attention layer.    
    Notes
    -----
    This implementation uses a single linear layer to project the input tensor
    into query, key, and value tensors. The attention mechanism is applied to
    these tensors, and the result is projected back to the original feature space.
    """

    def __init__(
            self,
            n_features: int # number of features
            ):
        super().__init__()
        self.scale = math.sqrt(n_features)

        # TODO: check why GroupNorm is used
        # self.norm = nn.GroupNorm(1, n_features)
        self.norm = nn.BatchNorm2d(n_features)

        # project to focus on specific channel areas
        # self.k_projection = nn.Linear(n_features, n_features)
        # self.q_projection = nn.Linear(n_features, n_features)
        # self.v_projection = nn.Linear(n_features, n_features)

        # the 3 previous projections can be replaced by 1 larger projection
        self.kqv = nn.Linear(n_features, n_features*3)
        self.proj = nn.Linear(n_features, n_features)
        
    def forward(self, x: torch.Tensor):
        B, C, H, W = x.shape
        inp = x
        t = self.norm(x).view(B, C, -1).transpose(1, 2) # B, T=H*W, C
        # k = self.k_projection(t)
        # q = self.q_projection(t)
        # v = self.v_projection(t)
        kqv = self.kqv(t)
        q, k, v = kqv.chunk(3, dim=-1)
        s = (q@k.transpose(1,2))/self.scale
        x = s.softmax(dim=-1)@v
        x = self.proj(x)
        x = x.transpose(1, 2).view(B, C, H, W)
        return x + inp #specific to stable diffusion ~resnet style

In [ ]:
show_doc(SelfAttention, title_level=3)

---

### SelfAttention

>      SelfAttention (n_features:int)

*Self-Attention layer for image data.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| n_features | int | number of features |

In [ ]:
B, C, H, W = 64, 3, 16, 16
x = torch.randn(B, C, H, W)
sa = SelfAttention(3)
sa(x).shape

torch.Size([64, 3, 16, 16])

In [ ]:
#| export

class MultiheadAttention(nn.Module):
    """
    Multihead Attention module.
    
    Parameters
    ----------
    n_features : int
        Number of features.
    n_heads : int
        Number of heads or groups of features.
    Methods
    -------
    forward(x: torch.Tensor) -> torch.Tensor
        Forward pass of the multihead attention module.
    """

    def __init__(
            self,
            n_features: int, #number of features
            n_heads: int #number of heads or groups of features
            ):
        
        super().__init__()
        self.n_heads = n_heads # number of heads
        self.scale = math.sqrt(n_features)
        self.norm = nn.BatchNorm2d(n_features)
        self.kqv = nn.Linear(n_features, n_features*3)
        self.proj = nn.Linear(n_features, n_features)

    def forward(self, x: torch.Tensor)->torch.Tensor:
        B, C, H, W = x.shape
        inp = x
        t = self.norm(x).view(B, C, -1).transpose(1, 2) # B, T=H*W, C
        kqv = self.kqv(t)
        # split channels into n_heads and reshape
        kqv = rearrange(kqv, 'b t (h d) -> (b h) t d', h=self.n_heads)
        q, k, v = kqv.chunk(3, dim=-1)
        s = (q@k.transpose(1,2))/self.scale
        x = s.softmax(dim=-1)@v
        x = rearrange(x, '(b h) t d -> b t (h d)', h=self.n_heads)
        x = self.proj(x)
        x = x.transpose(1, 2).view(B, C, H, W)
        return x + inp #specific to stable diffusion ~resnet style
        
    

In [ ]:
show_doc(MultiheadAttention, title_level=3)

---

### MultiheadAttention

>      MultiheadAttention (n_features:int, n_heads:int)

*Multihead Attention module.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| n_features | int | number of features |
| n_heads | int | number of heads or groups of features |

In [ ]:
B, C, H, W = 64, 32, 16, 16
x = torch.randn(B, C, H, W)
mha = MultiheadAttention(32, 8)
mha(x).shape

torch.Size([64, 32, 16, 16])

In [ ]:
nm = nn.MultiheadAttention(32, num_heads=8, batch_first=True)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()